In [ ]:
#%pylab inline

In [1]:
import numpy as np
import os
import pandas as pd
import sys
import time

from pandas import DataFrame, TimedeltaIndex, Series
from pandas.tslib import Timestamp
#from itertools import chain
#from progressbar import ProgressBar

In [2]:
### set paths to lookup data ###

MIMIC_PATH = '/Users/Mark/Downloads/MIMIC Data/Original'
VAR_BOUNDS_FILE = '/Users/Mark/Documents/GitHub/MIMIC-Machine-Learning/Reference Files/mimic3_variables.csv'
OUTPUT_PATH = '/Users/Mark/Downloads/MIMIC Data/Processed/Episodes'

try:
    os.makedirs(OUTPUT_PATH)
except:
    pass

In [3]:
var_bounds = DataFrame.from_csv(VAR_BOUNDS_FILE, index_col='variable')

In [4]:
st = time.time()
sys.stdout.write('Loading data...')
sys.stdout.flush()
ndata = np.load(os.path.join(OUTPUT_PATH, 'mimic3_12.npz'))
X = ndata['X']
X_names = ndata['X_names']

episode_id = ndata['episode_id']
admit_id = ndata['admit_id']
patient_id = ndata['patient_id']
encounter_no = ndata['encounter_no']
age = ndata['age']
race = ndata['race']
sex = ndata['sex']
admit_diagnosis = ndata['admit_diagnosis']
race_categories = ndata['race_categories']
sex_categories = ndata['sex_categories']
admit_diagnoses = ndata['admit_diagnoses']

ylos = ndata['ylos']

print('Took {0} secs'.format(time.time()-st))

Loading data...Took 15.408013343811035 secs


In [7]:
### do some additional data processing ###

X_hr = []
Ximp_hr = []

#bar = ProgressBar()
st = time.time()
#for xid in bar(range(X.shape[0])):
nb_samp = X.shape[0]
for xid in range(nb_samp):
    sys.stdout.write('\rProcessing sample {0} of {1}...'.format(xid, nb_samp))
    # timestamps
    x = DataFrame(X[xid], columns=X_names)
    x.time = TimedeltaIndex(x.time, 'm')
    x.set_index('time', inplace=True)
    
    # create hourly resampled
    x_hr = x.resample('1H', how='mean', closed='left')
    assert(x_hr.index[0].total_seconds() == 0)
    ximp_hr = x_hr.combine_first(x.resample('1H', how='last', fill_method='ffill', closed='left'))
    assert(ximp_hr.index[0].total_seconds() == 0)
    for c in np.intersect1d(ximp_hr.columns, var_bounds.index):
        v = ximp_hr[c]
        v[v.isnull()] = var_bounds.imputeValue[c]
        ximp_hr[c] = v
    assert(ximp_hr.notnull().values.any())
    
    X_hr.append(x_hr.values)
    Ximp_hr.append(ximp_hr.values)

print('\nTook', time.time()-st)

Processing data...
Processing sample 45881 of 45882...Took 1005.6535608768463


C:\Users\Mark\Miniconda3\lib\site-packages\ipykernel\__main__.py:19: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
C:\Users\Mark\Miniconda3\lib\site-packages\ipykernel\__main__.py:21: FutureWarning: fill_method is deprecated to .resample()
the new syntax is .resample(...).last().ffill()


In [11]:
fns = [ os.path.join(OUTPUT_PATH, 'mimic3_11_resampled.npz'),
       os.path.join(OUTPUT_PATH, 'mimic3_11_resampled-imputed.npz')
     ]
sequences = [ X_hr, Ximp_hr ]
for fn, seq in zip(fns, sequences):
    sys.stdout.write(fn + '...')
    sys.stdout.flush()
    st = time.time()
    np.savez(fn,X=seq,
                episode_id=episode_id,
                admit_id=admit_id,
                patient_id=patient_id,
                encounter_no=encounter_no,
                age=age,
                race=race,
                sex=sex,
                admit_diagnosis=admit_diagnosis,
                ylos=ylos,
                race_categories=race_categories,
                sex_categories=sex_categories,
                admit_diagnoses=admit_diagnoses,
                X_names=X_names[1:])
    print('DONE! Took', time.time()-st, 'secs')

/Users/Mark/Downloads/MIMIC Data/Processed/Episodes\mimic3_11_resampled.npz...DONE! Took 57.641146421432495 secs
/Users/Mark/Downloads/MIMIC Data/Processed/Episodes\mimic3_11_resampled-imputed.npz...DONE! Took 59.80550670623779 secs
